### Установка JVM

In [ ]:
%%capture
!sudo apt-get update
!sudo apt-get upgrade
!sudo apt-get install default-jre
#!java -version

### Установка Apache Spark

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3-scala2.13.tgz
!tar -xzf /content/spark-3.4.1-bin-hadoop3-scala2.13.tgz
!mv /content/spark-3.4.1-bin-hadoop3-scala2.13 /content/spark
!rm /content/spark-3.4.1-bin-hadoop3-scala2.13.tgz

### Проверка версии Apache Spark и Scala

In [ ]:
!/content/spark/bin/spark-shell --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.13.8, OpenJDK 64-Bit Server VM, 11.0.20.1
Branch HEAD
Compiled by user centos on 2023-06-19T22:21:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


### Создание структуры папок/файлов проекта

In [ ]:
!mkdir -p /content/SparkProject/src/{main,test}/{java,resources,scala}
!mkdir /content/SparkProject/{project,target}

In [ ]:
!touch /content/SparkProject/build.sbt
!touch /content/SparkProject/src/main/scala/SimpleApp.scala

### Заполнение файлов build.sbt и SimpleApp.scala

**Содержимое файла build.sbt**

```
name := "Simple Project"

version := "1.0"

scalaVersion := "2.13.8"

val sparkVersion = "3.4.1"

libraryDependencies ++= Seq(
  "org.apache.spark" %% "spark-core" % sparkVersion,
  "org.apache.spark" %% "spark-sql" % sparkVersion
)
```

**Содержимое файла SimpleApp.scala**
```
import org.apache.spark.sql.SparkSession

object SimpleApp {
  def main(args: Array[String]) {
    val logFile = "/content/spark/README.md"
    val spark = SparkSession.builder
                           .master("local[*]")
                           .appName("Simple Application")
                           .getOrCreate()
    val logData = spark.read.textFile(logFile).cache()
    val numAs = logData.filter(line => line.contains("a")).count()
    val numBs = logData.filter(line => line.contains("b")).count()
    println("*****")
    println(s"Lines with a: $numAs, Lines with b: $numBs")
    println("*****")
    spark.stop()
  }
}
```

### Установка SBT

In [ ]:
%%capture
!echo "deb https://repo.scala-sbt.org/scalasbt/debian all main" | sudo tee /etc/apt/sources.list.d/sbt.list
!echo "deb https://repo.scala-sbt.org/scalasbt/debian /" | sudo tee /etc/apt/sources.list.d/sbt_old.list
!curl -sL "https://keyserver.ubuntu.com/pks/lookup?op=get&search=0x2EE0EA64E40A89B84B2DF73499E82A75642AC823" | sudo apt-key add
!sudo apt-get update
!sudo apt-get install sbt

### Смена рабочей папки

In [ ]:
%cd /content/SparkProject

/content/SparkProject


### Сборка

In [ ]:
!sbt package

### Вывод результата

In [ ]:
!/content/spark/bin/spark-submit \
                --class "SimpleApp" \
                --master local[*] \
                /content/SparkProject/target/scala-2.13/simple-project_2.13-1.0.jar